In [1]:
import ROOT as r
import math
import array
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages
import uproot
import numpy as np
from scipy.optimize import curve_fit

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, 'include')
sys.path.insert(0, include_dir)
#__________________________________________________

import CONFIG
import DBPARSE
from UTILITIES import *
from SIMFITS2D import DistributionFits2D
from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend

Welcome to JupyROOT 6.26/06


In [2]:
Pass="1"

In [3]:
GEN2Path=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass{Pass}/Pass{Pass}_data_GEN2_sbs100p_nucleon_np_model2.root"
GEN3Path=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass{Pass}/Pass{Pass}_data_GEN3_sbs100p_nucleon_np_model2.root"
GEN4aPath=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass{Pass}/Pass{Pass}_data_GEN4a_sbs100p_nucleon_np_model2.root"
GEN4bPath=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass{Pass}/Pass{Pass}_data_GEN4b_sbs100p_nucleon_np_model2.root"

In [8]:
#available branches
import ROOT as r


def list_branches(rootfile_path, tree_name="Tout"):
    # Open the ROOT file
    root_file = r.TFile.Open(rootfile_path)
    
    # Check if the file was successfully opened
    if not root_file or root_file.IsZombie():
        print(f"Error: Could not open ROOT file '{rootfile_path}'.")
        return
    
    # Access the TTree
    tree = root_file.Get(tree_name)
    
    # Check if the tree exists
    if not tree:
        print(f"Error: Tree '{tree_name}' not found in file '{rootfile_path}'.")
        root_file.Close()
        return
    
    # Get the list of branches
    branch_list = tree.GetListOfBranches()
    
    # Print all branch names
    print(f"Branches in tree '{tree_name}':")
    for branch in branch_list:
        print(branch.GetName())
    
    # Close the ROOT file
    root_file.Close()

# Example usage
rootfile_path = "/home/research/GEn/GEn Data Analysis/outfiles/Pass1/QE_Insim_GEN3_sbs100p_nucleon_np_model2Zheng.root"
list_branches("/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN3_sbs100p_nucleon_np_model2.root")


Branches in tree 'Tout':
runnum
WCut
pCut
nCut
fiduCut
coinCut
ebeam
nu
Q2
W2
dpel
ephi
etheta
pcentral
vz
vx
vy
xtgt
ytgt
thtgt
phtgt
thetabend
xfp
yfp
thfp
phfp
trP
trPx
trPy
trPz
ePS
xPS
eSH
xSH
ySH
eHCAL
xHCAL
yHCAL
xHCAL_exp
yHCAL_exp
dx
dy
ngrinch_hits
xGRINCH
yGRINCH
coin_time
hcal_time
bbcal_time
nhodo_clus
hodo_time
BPMAx
BPMAy
Rasterx
Rastery
Raster2x
Raster2y
helicity
IHWP


In [9]:
"""
#inputs hodo
config="3"
prefix="HCal_data"
detector="hodo"
#inputs hcal
config="3"
prefix="HCal_data"
detector="hcal"
"""

'\n#inputs hodo\nconfig="3"\nprefix="HCal_data"\ndetector="hodo"\n#inputs hcal\nconfig="3"\nprefix="HCal_data"\ndetector="hcal"\n'

In [13]:
#define branches
branches=["ebeam","nu","Q2","W2","dpel","ephi","etheta","pcentral","vz","vx"
          ,"vy","xtgt","ytgt","thtgt","phtgt","thetabend","xfp","yfp","thfp"
          ,"phfp","trP","trPx","trPy","trPz","ePS","xPS","eSH","xSH","ySH","eHCAL"
          ,"xHCAL","yHCAL","xHCAL_exp","yHCAL_exp","dx","dy","ngrinch_hits","xGRINCH"
          ,"yGRINCH","coin_time","hcal_time","bbcal_time","BPMAx","BPMAy","Rasterx"
          ,"Rastery","Raster2x","Raster2y","helicity","IHWP","pblkid","tdc","atime"
          ,"nblk","mag","runnum","tar","cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime","hodoTimes","hodoIDs","hodoTOT","nbars"]
hodo_branches=["hodoTimes","hodoIDs","hodoTOT","nbars","hodoClusMean"]
hcal_branches=["cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime"]
trigger_branches=["trigger","triggerID","NdataTriggerID"]
generic_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","thtgt","phtgt",
                  "coin_time","trPx","trPy","trPz","ephi","ebeam","helicity","IHWP","runnum"]

sim_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","fnucl","weight"]
insim_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","trP","fnucl","weight","mc_sig"]


ml_branches = [
    "ebeam", "nu", "Q2", "W2", "dpel", "ephi", "etheta", "pcentral", "vz", "vx", "vy",
    "xtgt", "ytgt", "thtgt", "phtgt", "xfp", "yfp", "thfp", "phfp", "trP",
    "ePS", "eSH", "eHCAL", "xHCAL", "yHCAL", "xHCAL_exp", "yHCAL_exp",
    "dx", "dy"
]


In [5]:

def ConvertToNP(branches,config,prefix,detector,File):
    # Open the ROOT file
    with uproot.open(File) as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)

        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpyData/Pass{Pass}/GEN{config}/{prefix}/{detector}{j}.npz", **data)


In [15]:
def ConvertToNPSim(branches,config,prefix,detector,File):
    # Open the ROOT file
    with uproot.open(f"outfiles/Pass1/QE_sim_GEN3_sbs100p_nucleon_np_model2.root") as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


In [20]:
def ConvertToNPInSim(branches,config,prefix,detector):
    # Open the ROOT file
    with uproot.open(f"outfiles/Pass1/QE_Insim_GEN{config}_sbs100p_nucleon_np_model2Zheng.root") as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


In [9]:
#Hodoscope
ConvertToNP(hodo_branches,"3","HCal_data","hodo")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [ ]:
#HCal
ConvertToNP(hcal_branches,"3","HCal_data","hcal")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13


In [ ]:
#Trigger
ConvertToNP(trigger_branches,"3","HCal_data","trigger")

In [14]:
ConvertToNP(generic_branches,"2","He3","generic",GEN2Path)

Processing Chunk 1 out of 14
Processing Chunk 2 out of 14
Processing Chunk 3 out of 14
Processing Chunk 4 out of 14
Processing Chunk 5 out of 14
Processing Chunk 6 out of 14
Processing Chunk 7 out of 14
Processing Chunk 8 out of 14
Processing Chunk 9 out of 14
Processing Chunk 10 out of 14
Processing Chunk 11 out of 14
Processing Chunk 12 out of 14
Processing Chunk 13 out of 14
Processing Chunk 14 out of 14


In [13]:
#Generic
ConvertToNP(generic_branches,"4a","He3","generic",GEN4aPath)

Processing Chunk 1 out of 5
Processing Chunk 2 out of 5
Processing Chunk 3 out of 5
Processing Chunk 4 out of 5
Processing Chunk 5 out of 5


In [22]:
ConvertToNP(generic_branches,"4b","He3","generic",GEN4bPath)

Processing Chunk 1 out of 1


In [12]:
ConvertToNP(generic_branches,"3","He3","generic",GEN3Path)

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [ ]:
#MachineLearning Data/ElasticSim/InelasticSim
ConvertToNP(ml_branches,"3","HCal_data","machinelearningD")
ConvertToNPSim(ml_branches,"3","HCal_data","machinelearningES")
ConvertToNPInSim(ml_branches,"3","HCal_data","machinelearningIS")

In [16]:
#Simulation
ConvertToNPSim(sim_branches,"3","HCal_data","simulation",3)

264763
Processing Chunk 1 out of 1


In [29]:
#Simulation
ConvertToNPInSim(insim_branches,"3","HCal_data","insimulation")

58917
Processing Chunk 1 out of 1
